In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [22]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz


In [ ]:
!tar xvzf spark-3.2.1-bin-hadoop3.2.tgz

In [48]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [27]:
!pip install findspark

In [51]:
!echo $SPARK_HOME

/content/spark-3.2.1-bin-hadoop3.2


In [3]:
import findspark
findspark.init('/home/al/spark-3.1.1-bin-hadoop3.2')

import os
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import  SparkContext
from pyspark.sql import functions as f
appName = "PySpark Task 1"
#master = "spark://10.3.100.4:7077"
master = 'local[*]'
# Create Spark session with Hive supported.
spark = SparkSession.builder \
    .enableHiveSupport() \
    .appName(appName) \
    .getOrCreate()

23/09/06 22:04:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
#подгрузим все датасеты по порядку и выведем их схемы и первые пару строк
df_countr = spark.read.orc("data_for_exam/countries")
df_countr.printSchema()
df_countr.show(2)

root
 |-- COUNTRY_ID: string (nullable = true)
 |-- COUNTRY_NAME: string (nullable = true)
 |-- REGION_ID: integer (nullable = true)



+----------+------------+---------+
|COUNTRY_ID|COUNTRY_NAME|REGION_ID|
+----------+------------+---------+
|        AR|   Argentina|        2|
|        AU|   Australia|        3|
+----------+------------+---------+
only showing top 2 rows



In [5]:
df_dep = spark.read\
.option("sep", ",")\
.option("header", "True")\
.option("inferSchema", True)\
.csv("data_for_exam/departments/departments")
df_dep.printSchema()
df_dep.show(2)

root
 |-- DEPARTMENT_ID: integer (nullable = true)
 |-- DEPARTMENT_NAME: string (nullable = true)
 |-- MANAGER_ID: integer (nullable = true)
 |-- LOCATION_ID: integer (nullable = true)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
|           20|      Marketing|       201|       1800|
+-------------+---------------+----------+-----------+
only showing top 2 rows



In [6]:
df_emp = spark.read\
.option("sep", "\t")\
.option("header", "True")\
.option("inferSchema", True)\
.csv("data_for_exam/employees/employees")
df_emp.printSchema()
df_emp.show(2)

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: integer (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)

+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567| 17.06.03|AD_PRES| 24000|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|51

In [7]:
df_jhis = spark.read.json("data_for_exam/job_history/job_history")
df_jhis.printSchema()
df_jhis.show(2)

root
 |-- department_id: long (nullable = true)
 |-- employee_id: long (nullable = true)
 |-- end_date: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- start_date: string (nullable = true)

+-------------+-----------+--------+----------+----------+
|department_id|employee_id|end_date|    job_id|start_date|
+-------------+-----------+--------+----------+----------+
|           60|        102|24.07.06|   IT_PROG|  13.01.01|
|          110|        101|27.10.01|AC_ACCOUNT|  21.09.97|
+-------------+-----------+--------+----------+----------+
only showing top 2 rows



In [8]:
#df_emp = spark.read.json("/content/sample_data/anscombe.json")
df_jobs = spark.read\
.option("sep", ";")\
.option("header", "True")\
.option("inferSchema", True)\
.csv("data_for_exam/jobs/jobs")
df_jobs.printSchema()
df_jobs.show(2)

root
 |-- JOB_ID: string (nullable = true)
 |-- JOB_TITLE: string (nullable = true)
 |-- MIN_SALARY: integer (nullable = true)
 |-- MAX_SALARY: integer (nullable = true)

+-------+--------------------+----------+----------+
| JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|
+-------+--------------------+----------+----------+
|AD_PRES|           President|     20080|     40000|
|  AD_VP|Administration Vi...|     15000|     30000|
+-------+--------------------+----------+----------+
only showing top 2 rows



In [9]:
df_loc = spark.read.parquet("data_for_exam/locations")
df_loc.printSchema()
df_loc.show(2)

root
 |-- LOCATION_ID: integer (nullable = true)
 |-- STREET_ADDRESS: string (nullable = true)
 |-- POSTAL_CODE: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE_PROVINCE: string (nullable = true)
 |-- COUNTRY_ID: string (nullable = true)



+-----------+--------------------+-----------+------+--------------+----------+
|LOCATION_ID|      STREET_ADDRESS|POSTAL_CODE|  CITY|STATE_PROVINCE|COUNTRY_ID|
+-----------+--------------------+-----------+------+--------------+----------+
|       1000|1297 Via Cola di Rie|      00989|  Roma|          null|        IT|
|       1100|93091 Calle della...|      10934|Venice|          null|        IT|
+-----------+--------------------+-----------+------+--------------+----------+
only showing top 2 rows



In [ ]:
#df_reg = 

In [12]:
df_emp.createOrReplaceTempView("employees")
df_countr.createOrReplaceTempView("countries")
df_loc.createOrReplaceTempView("locations")
df_dep.createOrReplaceTempView("departments")

In [22]:
#1
df = spark.sql("""SELECT last_name || ' зарабатывает ' || salary || ' каждый месяц, но хочет получать ' || salary*3
                  AS Dream_Salaries
                  FROM employees""")

df.write\
.format("parquet")\
.option("compression", "gzip")\
.mode("overwrite")\
.save("results/task1")

spark.read.format("parquet")\
.option("compression", "gzip")\
.load("results/task1")\
.show(5, False)

+----------------------------------------------------------------+
|Dream_Salaries                                                  |
+----------------------------------------------------------------+
|King зарабатывает 24000 каждый месяц, но хочет получать 72000   |
|Kochhar зарабатывает 17000 каждый месяц, но хочет получать 51000|
|De Haan зарабатывает 17000 каждый месяц, но хочет получать 51000|
|Hunold зарабатывает 9000 каждый месяц, но хочет получать 27000  |
|Ernst зарабатывает 6000 каждый месяц, но хочет получать 18000   |
+----------------------------------------------------------------+
only showing top 5 rows



In [48]:
#2
df = spark.sql("""SELECT location_id, street_address, city, state_province, country_name
                  FROM locations
                  JOIN countries ON locations.country_id = countries.country_id""")
df.write\
.format("parquet")\
.option("compression", "gzip")\
.mode("overwrite")\
.save("results/task2")

spark.read.format("parquet")\
.option("compression", "gzip")\
.load("results/task2")\
.show(5, False)

+-----------+-----------------------+---------+----------------+------------------------+
|location_id|street_address         |city     |state_province  |country_name            |
+-----------+-----------------------+---------+----------------+------------------------+
|1000       |1297 Via Cola di Rie   |Roma     |null            |Italy                   |
|1100       |93091 Calle della Testa|Venice   |null            |Italy                   |
|1200       |2017 Shinjuku-ku       |Tokyo    |Tokyo Prefecture|Japan                   |
|1300       |9450 Kamiya-cho        |Hiroshima|null            |Japan                   |
|1400       |2014 Jabberwocky Rd    |Southlake|Texas           |United States of America|
+-----------+-----------------------+---------+----------------+------------------------+
only showing top 5 rows



In [13]:
#3
df = spark.sql("""SELECT last_name, departments.department_id, department_name
                  FROM employees
                  JOIN departments ON employees.department_id = departments.department_id""")
df.show(5)

#сохранить в формате avro со сжатием gzip

+---------+-------------+---------------+
|last_name|department_id|department_name|
+---------+-------------+---------------+
|     King|           90|      Executive|
|  Kochhar|           90|      Executive|
|  De Haan|           90|      Executive|
|   Hunold|           60|             IT|
|    Ernst|           60|             IT|
+---------+-------------+---------------+
only showing top 5 rows



In [39]:
#4
df = spark.sql("""SELECT empl.last_name AS Employee,
                empl.employee_id AS EMP,
                chief.last_name AS Manager,
                empl.manager_id AS Mgr
                FROM employees AS empl
                LEFT JOIN employees AS chief ON chief.employee_id = empl.manager_id""")
df.show(5)

#Результат сохранить в формате avro со сжатием Snappy

+--------+---+-------+----+
|Employee|EMP|Manager| Mgr|
+--------+---+-------+----+
|    King|100|   null|null|
| Kochhar|101|   King| 100|
| De Haan|102|   King| 100|
|  Hunold|103|De Haan| 102|
|   Ernst|104| Hunold| 103|
+--------+---+-------+----+
only showing top 5 rows



In [49]:
#5
df = spark.sql("""SELECT last_name, salary
                FROM employees
                WHERE manager_id in (SELECT employee_id
                                    FROM employees
                                    WHERE last_name = 'King')""")
df.write\
.format("parquet")\
.option("compression", "gzip")\
.mode("overwrite")\
.save("results/task5")

spark.read.format("parquet")\
.option("compression", "gzip")\
.load("results/task5")\
.show(5, False)

+---------+------+
|last_name|salary|
+---------+------+
|Kochhar  |17000 |
|De Haan  |17000 |
|Raphaely |11000 |
|Weiss    |8000  |
|Fripp    |8200  |
+---------+------+
only showing top 5 rows



In [50]:
#6
df = spark.sql("""SELECT last_name
                FROM employees
                WHERE salary > (SELECT max(salary)
                                FROM employees
                                WHERE department_id = 60)""")
df.write\
.format("parquet")\
.option("compression", "gzip")\
.mode("overwrite")\
.save("results/task6")

spark.read.format("parquet")\
.option("compression", "gzip")\
.load("results/task6")\
.show(5, False)

+---------+
|last_name|
+---------+
|King     |
|Kochhar  |
|De Haan  |
|Greenberg|
|Raphaely |
+---------+
only showing top 5 rows



In [18]:
#7
df = spark.sql("""SELECT employee_id, last_name, salary
                FROM employees
                WHERE department_id in (SELECT department_id
                                        FROM employees
                                        WHERE upper(last_name) LIKE '%U%')
                AND salary > (SELECT avg(salary)
                              FROM employees)""")
df.write\
.format("parquet")\
.option("compression", "gzip")\
.mode("overwrite")\
.save("results/task7")

spark.read.format("parquet")\
.option("compression", "gzip")\
.load("results/task7")\
.show(5, False)

+-----------+---------+------+
|employee_id|last_name|salary|
+-----------+---------+------+
|103        |Hunold   |9000  |
|108        |Greenberg|12008 |
|109        |Faviet   |9000  |
|110        |Chen     |8200  |
|111        |Sciarra  |7700  |
+-----------+---------+------+
only showing top 5 rows



In [52]:
#8
df = spark.sql("""
SELECT last_name, employees.department_id, department_name
FROM employees
LEFT JOIN departments
        ON employees.department_id = departments.department_id
WHERE employees.department_id IS NULL
    UNION
SELECT last_name, employees.department_id, department_name
FROM departments
LEFT JOIN employees
        ON employees.department_id = departments.department_id
WHERE employees.last_name IS NULL""")

df.write\
.format("parquet")\
.option("compression", "gzip")\
.mode("overwrite")\
.save("results/task8")

spark.read.format("parquet")\
.option("compression", "gzip")\
.load("results/task8")\
.show(5, False)

+---------+-------------+--------------------+
|last_name|department_id|department_name     |
+---------+-------------+--------------------+
|null     |null         |Shareholder Services|
|null     |null         |Control And Credit  |
|null     |null         |Government Sales    |
|null     |null         |Corporate Tax       |
|null     |null         |Manufacturing       |
+---------+-------------+--------------------+
only showing top 5 rows

